# Modellierung Windeinspeisung

**Lernziele**
- Hyperparameteroptimierung mit tensorboard

## Pakete und Daten laden

In [1]:
# Datenorganisation
import pandas as pd
import numpy as np
import datetime 

# Ploterstellung
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib widget
import seaborn as sns

# Datenvorbereitung
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split

# Standardeinstellungen
plt.rcParams['axes.xmargin'] = 0
pd.set_option('display.precision',3)
np.set_printoptions(precision=3)

# Tensorflow
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, LSTM, Dense, Bidirectional, Dropout)
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import (EarlyStopping, ModelCheckpoint)
from tensorflow import keras

import tensorflow as tf
import keras_tuner as kt
import tensorboard
from tensorboard.plugins.hparams import api as hp
tensorboard.__version__
# Load the TensorBoard notebook extension
%load_ext tensorboard

#sonst  Module 
import copy
import sys
from math import ceil

# Eingene Module 
# temporäre Einbinden des neuen Pfades mit zu ladenden Dateien
sys.path.append("C:\\Users\\rs3753e\\sciebo\\Vorlesungen\\SoSe_Energiedatenanalyse Datamining\\internalfunctions\\")
from KNN import KNN

In [2]:
# Laden des vorbereiteten WinddatenFrame
wind=pd.read_hdf('../daten/windeinspeisung_bereinigt','Obj1')

## Datenorganisation

Erstellung eines KNN-Objektes mit normierten/Standardisierten Trainings/Validierungs- und Testdaten

In [3]:
# Vorgaben zur Datentrennung
sections=(.6,.25,.15)
shuffle=False

# Vorbereitete Klasse
model = KNN()
# Unterteilung der Samples
model.split_sample(pd.DataFrame(wind["Ws_avg"]),pd.DataFrame(wind["S_avg"]),shuffle=shuffle,sections =sections)
#Auswahl der Skalierungsmethode
model.scaler("MinMax")

## Erstellung eines KNN zur Prognose der Einspeisung

**Definition Features und Output**

In [4]:
model.numFeatures = 1
model.numResponses = 1;
model.x[0].shape

(121071, 1)

### Hyperparameter definieren

Die Ausprägungen der Parametervariation wird festgelegt.
hp.HParam ist eine Klasse, um die Hyperparameter zu definieren und zu loggen. Die Klasse wird von tensorboard zur Verfügung gestellt.

### Modellaufbau mit Keras-functional API Schreibweise

In [13]:
# Modellaufbau mit gewählter Hyperparameterausstattung  
    # Define model layers.
input_layer = Input(shape=(model.numFeatures,))

X= Dense(units=128,activation = 'relu',name='dense1')(input_layer)
X= Dense(units=128,activation = 'relu',name='dense2')(X) 
#X = Dropout(0.1)(X)

# Output
y_output= Dense(units='1', name='output')(X)  

# Verbindung von Input und Output-Schicht   
KNN = Model(inputs=input_layer,outputs=y_output)

KNN.compile(optimizer=keras.optimizers.Adam(learning_rate =1E-1),
            loss=keras.losses.MeanSquaredError(),
            metrics=keras.losses.MeanAbsoluteError()) 

Der callback `tf.keras.callbacks.TensorBoard` ermöglicht es, dass der Lauf in einem entsprechenden Verzeichnis `log_dir` abgespeichert wird.

In [14]:
early = EarlyStopping(monitor='val_loss', patience=5)
tb_callback = tf.keras.callbacks.TensorBoard(log_dir = "logs_single_run/ohneDO",histogram_freq=1)

KNN.fit(x=model.x_norm[0], 
        y=model.y_norm[0], 
        epochs=6,
        validation_data=(model.x_norm[1], model.y_norm[1]), 
        callbacks=[
              early,
              tb_callback  # log metrics
                        ],
       )

Epoch 1/6
3784/3784 [==============================] - 13s 3ms/step - loss: 0.0538 - mean_absolute_error: 0.1709 - val_loss: 0.0510 - val_mean_absolute_error: 0.1564
Epoch 2/6
3784/3784 [==============================] - 12s 3ms/step - loss: 0.0490 - mean_absolute_error: 0.1698 - val_loss: 0.0493 - val_mean_absolute_error: 0.1598
Epoch 3/6
3784/3784 [==============================] - 12s 3ms/step - loss: 0.0490 - mean_absolute_error: 0.1698 - val_loss: 0.0544 - val_mean_absolute_error: 0.2001
Epoch 4/6
3784/3784 [==============================] - 12s 3ms/step - loss: 0.0490 - mean_absolute_error: 0.1699 - val_loss: 0.0484 - val_mean_absolute_error: 0.1718
Epoch 5/6
3784/3784 [==============================] - 12s 3ms/step - loss: 0.0490 - mean_absolute_error: 0.1696 - val_loss: 0.0494 - val_mean_absolute_error: 0.1801
Epoch 6/6
3784/3784 [==============================] - 12s 3ms/step - loss: 0.0489 - mean_absolute_error: 0.1697 - val_loss: 0.0483 - val_mean_absolute_error: 0.1691


In [10]:
%tensorboard --port 6006 --logdir logs_single4

A brief overview of the visualizations created in this example and the dashboards (tabs in top navigation bar) where they can be found:

* **Scalars** show how the loss and metrics change with every epoch. You can use them to also track training speed, learning rate, and other scalar values. Scalars can be found in the **Time Series** or **Scalars** dashboards.
* **Graphs** help you visualize your model. In this case, the Keras graph of layers is shown which can help you ensure it is built correctly. Graphs can be found in the **Graphs** dashboard.
* **Histograms** and **Distributions** show the distribution of a Tensor over time. This can be useful to visualize weights and biases and verify that they are changing in an expected way. Histograms can be found in the **Time Series** or **Histograms** dashboards. Distributions can be found in the **Distributions** dashboard.

Additional TensorBoard dashboards are automatically enabled when you log other types of data. For example, the Keras TensorBoard callback lets you log images and embeddings as well. You can see what other dashboards are available in TensorBoard by clicking on the "inactive" dropdown towards the top right.



The left pane of the dashboard provides filtering capabilities that are active across all the views in the HParams dashboard:

    Filter which hyperparameters/metrics are shown in the dashboard
    Filter which hyperparameter/metrics values are shown in the dashboard
    Filter on run status (running, success, ...)
    Sort by hyperparameter/metric in the table view
    Number of session groups to show (useful for performance when there are many experiments)



he HParams dashboard has three different views, with various useful information:

    The Table View lists the runs, their hyperparameters, and their metrics.
    The Parallel Coordinates View shows each run as a line going through an axis for each hyperparemeter and metric. Click and drag the mouse on any axis to mark a region which will highlight only the runs that pass through it. This can be useful for identifying which groups of hyperparameters are most important. The axes themselves can be re-ordered by dragging them.
    The Scatter Plot View shows plots comparing each hyperparameter/metric with each metric. This can help identify correlations. Click and drag to select a region in a specific plot and highlight those sessions across the other plots.

A table row, a parallel coordinates line, and a scatter plot market can be clicked to see a plot of the metrics as a function of training steps for that session (although in this tutorial only one step is used for each run).